# 1 degree-of-freedom example of derivative-free optimization involving VMEC

This script implements the "1DOF_circularCrossSection_varyR0_targetVolume" example from https://github.com/landreman/stellopt_scenarios . This optimization problem has one independent variable, representing the mean major radius. The problem also has one objective: the plasma volume. There is not actually any need to run an equilibrium code like VMEC since the objective function can be computed directly from the boundary shape. But this problem is a fast way to test the optimization infrastructure with VMEC.

Details of the optimum and a plot of the objective function landscape can be found here: https://github.com/landreman/stellopt_scenarios/tree/master/1DOF_circularCrossSection_varyR0_targetVolume 

In [1]:
import sys
sys.path.append('..')
import numpy as np
from simsopt import Vmec, LeastSquaresProblem, least_squares_serial_solve

Start with a default surface, which is axisymmetric with major radius 1 and minor radius 0.1. 

In [2]:
equil = Vmec()
surf = equil.boundary

Set the initial boundary shape. Here is one syntax for doing this:

In [3]:
surf.set('rc(0,0)', 1.0)

Here is another syntax that also works:

In [4]:
surf.set_rc(0, 1, 0.1)
surf.set_zs(0, 1, 0.1)

surf.set_rc(1, 0, 0.1)
surf.set_zs(1, 0, 0.1)

Now determine which parameters are varied in the optimization. VMEC parameters are all fixed by default, while surface parameters are all non-fixed by default. You can choose which parameters are optimized by setting their 'fixed' attributes.

In [5]:
surf.all_fixed()
surf.set_fixed('rc(0,0)', False)

Each function you want to optimize is then equipped with a shift and weight, to become a term in a least-squares objective function:

In [6]:
desired_volume = 0.15
weight = 1
term1 = (equil.volume, desired_volume, weight)

A list of terms are combined to form a nonlinear-least-squares problem:

In [7]:
prob = LeastSquaresProblem([term1])

Let's print out the initial global state vector, i.e. the vector of variables that is optimized. Each entry in this state vector has an associated string, explaining its meaning.

In [8]:
print(prob.x)
print(prob.dofs.names)

[1.]
['rc(0,0) of SurfaceRZFourier 0x10d260b20 (nfp=5, stelsym=True, mpol=5, ntor=4)']


Simsopt detects that gradient information is not available:

In [9]:
prob.dofs.grad_avail

False

Finally, let's solve the optimization problem. Simsopt detects that analytic derivatives are not available, and so chooses a derivative-free algorithm. During the optimization, VMEC's output is printed to the terminal window running the Jupyter server, not directly in this notebook. Here we use the serial solver `least_squares_serial_solve` to avoid the complication of using jupyter with MPI, but an MPI solver using parallelized finite-difference gradients is also available.

In [10]:
least_squares_serial_solve(prob)

Using derivative-free method
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.1230e-03                                    9.35e-03    
       1              2         6.9184e-18      1.12e-03       2.40e-01       7.34e-10    
`gtol` termination condition is satisfied.
Function evaluations 2, initial cost 1.1230e-03, final cost 6.9184e-18, first-order optimality 7.34e-10.


Let's examine the optimum:

In [11]:
print("At the optimum,")
print(" rc(m=0,n=0) = ", surf.get_rc(0, 0))
print(" volume, according to VMEC    = ", equil.volume())
print(" volume, according to Surface = ", surf.volume())
print(" objective function = ", prob.objective())

At the optimum,
 rc(m=0,n=0) =  0.7599088584729224
 volume, according to VMEC    =  0.14999999628022323
 volume, according to Surface =  0.1499999962802224
 objective function =  1.3836739176882137e-17


This solution matches the description in https://github.com/landreman/stellopt_scenarios/tree/master/1DOF_circularCrossSection_varyR0_targetVolume . We can do some asserts to be sure:

In [12]:
assert np.abs(surf.get_rc(0, 0) - 0.7599088773175) < 1.0e-5
assert np.abs(equil.volume() - 0.15) < 1.0e-6
assert np.abs(surf.volume() - 0.15) < 1.0e-6
assert prob.objective() < 1.0e-15